In [2]:
import pandas as pd

In [18]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

/home/codespace/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
feb_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
jan_df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [6]:
df_columns = jan_df.shape[1]
df_columns

19

In [7]:
jan_df['duration'] = jan_df["tpep_dropoff_datetime"] - jan_df["tpep_pickup_datetime"]
jan_df['duration'] = jan_df.duration.apply(lambda td: td.total_seconds() / 60)
jan_sd = jan_df['duration'].std()
# df = jan_df[(Jjn_df.duration >= 1) & (jan_df.duration <= 60)]
jan_filtered = jan_df[(jan_df.duration >= 1) & (jan_df.duration <= 60)]
# categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']
jan_sd
# df[categorical] = df[categorical].astype(str)

42.84210176105113

In [8]:
jan_df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.40,3.50,0.5,0.00,0.0,1.0,9.40,2.5,0.00,1.683333
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.00,-1.00,-0.5,0.00,0.0,-1.0,-5.50,0.0,0.00,0.233333
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.00,1.00,0.5,0.00,0.0,1.0,5.50,0.0,0.00,0.233333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.90,2.25,0.5,0.00,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.00,1.00,0.5,3.30,0.0,1.0,25.30,2.5,0.00,13.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2,2023-02-28 23:46:00,2023-03-01 00:05:00,NaN,4.65,NaN,None,249,140,0,20.22,0.00,0.5,4.84,0.0,1.0,29.06,NaN,NaN,19.000000
2913951,2,2023-02-28 23:26:02,2023-02-28 23:37:10,NaN,2.47,NaN,None,186,79,0,13.66,0.00,0.5,2.65,0.0,1.0,20.31,NaN,NaN,11.133333
2913952,2,2023-02-28 23:24:00,2023-02-28 23:38:00,NaN,3.49,NaN,None,158,143,0,17.64,0.00,0.5,0.00,0.0,1.0,21.64,NaN,NaN,14.000000
2913953,2,2023-02-28 23:03:00,2023-02-28 23:10:00,NaN,2.13,NaN,None,79,162,0,13.56,0.00,0.5,2.63,0.0,1.0,20.19,NaN,NaN,7.000000


In [9]:
fraction_remaining = len(jan_filtered) / len(jan_df)
fraction_remaining

0.9800944077722545

In [10]:
jan_filtered['PULocationID'] = jan_filtered['PULocationID'].astype(str)
jan_filtered['DOLocationID'] = jan_filtered['DOLocationID'].astype(str)
train_dicts = jan_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')
dv = DictVectorizer(sparse=True)
feature_matrix = dv.fit_transform(train_dicts)

/tmp/ipykernel_11478/1178262964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_filtered['PULocationID'] = jan_filtered['PULocationID'].astype(str)
/tmp/ipykernel_11478/1178262964.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_filtered['DOLocationID'] = jan_filtered['DOLocationID'].astype(str)


In [11]:
num_columns = feature_matrix.shape[1]

In [12]:
num_columns

514

In [13]:
target = jan_filtered['duration'].values
model = LinearRegression()
model.fit(feature_matrix, target)

LinearRegression()

In [14]:
train_predictions = model.predict(feature_matrix)
rmse_train = np.sqrt(mean_squared_error(target, train_predictions))

In [15]:
rmse_train

7.778947591690304

In [16]:
feb_df['duration'] = feb_df["tpep_dropoff_datetime"]- feb_df["tpep_dropoff_datetime"]
feb_df.duration = feb_df.duration.apply(lambda td: td.total_seconds() / 60)
feb_filtered = feb_df
feb_filtered['PULocationID'] = feb_filtered['PULocationID'].astype(str)
feb_filtered['DOLocationID'] = feb_filtered['DOLocationID'].astype(str)
feb_data_dicts = feb_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')
if feb_filtered.empty:
    print("February filtered data is empty.")
else:
    feb_filtered['PULocationID'] = feb_filtered['PULocationID'].astype(str)
    feb_filtered['DOLocationID'] = feb_filtered['DOLocationID'].astype(str)
    feb_data_dicts = feb_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

    # Transform February data using the dictionary vectorizer
    feb_feature_matrix = dv.transform(feb_data_dicts)

    # Calculate the RMSE on the validation data
    feb_target = feb_filtered['duration'].values
    val_predictions = model.predict(feb_feature_matrix)
    rmse_val = np.sqrt(mean_squared_error(feb_target, val_predictions))

    # Print results
    print("Number of columns in the feature matrix:", num_columns)
    print("RMSE on training data:", rmse_train)
    print("RMSE on validation data:", rmse_val)

Number of columns in the feature matrix: 514
RMSE on training data: 7.778947591690304
RMSE on validation data: 16.14772251293252
